In [242]:
set.seed(123)
###define a function to draw samples from a Dirichlet distribution
rDirichlet <- function(alpha_vec){
num <- length(alpha_vec)
temp <- rgamma(num, shape = alpha_vec, rate = 1)#shape 和rate是两个参数，不决定维度
return(temp / sum(temp))
}
n <- 90 #number of samples
n1 <- 30 #number of controls
n2 <- 60 #number of cases
m <- 400 #number of CpG sites
K <- 3 #underlying cell type number
###simulate methylation baseline profiles
#assume cell type 1 and cell type 2 are from the same lineage
#cell type 1
methy1 <- rbeta(m,3,6)#beta distribution,a=3,b=6
head(methy1)
length(methy1)
#cell type 2
methy2 <- methy1 + rnorm(m, sd=0.01)
ind <- sample(1:m, m/5)
methy2[ind] <- rbeta(length(ind),3,6)
#cell type 3
methy3 <- rbeta(m,3,6)
mu <- cbind(methy1, methy2, methy3)#(2000,3)
dim(mu)
#number of covariates
p <- 2#phenotype
###simulate covariates / phenotype (disease status and age)
X <- rbind(c(rep(0, n1),rep(1, n2)), runif(n, min=20, max=50))
X#(2,180)
###simulate phenotype effects
beta <- array(0, dim=c(m,K,p))
dim(beta)#(m,k,p)
#control vs case
m_common <- 10
max_signal <- 0.15
min_signal <- 0.07
#we allow different signs and magnitudes
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
signs
beta[1:m_common,1:K,1] <- signs * runif(m_common*K, min=min_signal, max=max_signal)
beta[1:m_common,1:K,1]
#这里是共同影响的点位，下面是不共同影响
m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta[m_common+(1:m_seperate),1:2,1] <- signs *runif(m_seperate*2, min=min_signal, max=max_signal)
#这里是1：2，下面是3，对应前面cell type1，2 from the same lineage. 与3不同
signs <- sample(c(-1,1), m_seperate, replace=TRUE)
beta[m_common+m_seperate+(1:m_seperate),K,1] <- signs *runif(m_seperate, min=min_signal, max=max_signal)
head(mu)

base <- 20
m_common <- 10
max_signal <- 0.015
min_signal <- 0.007
signs <- sample(c(-1,1), m_common*K, replace=TRUE)
beta[base+1:m_common,1:K,2] <- signs *
runif(m_common*K, min=min_signal, max=max_signal)
m_seperate <- 10
signs <- sample(c(-1,1), m_seperate*2, replace=TRUE)
beta[base+m_common+(1:m_seperate),1:2,2] <- signs *
runif(m_seperate*2, min=min_signal, max=max_signal)
signs <- sample(c(-1,1), m_seperate, replace=TRUE)
beta[base+m_common+m_seperate+(1:m_seperate),K,2] <- signs *
runif(m_seperate, min=min_signal, max=max_signal)
head(beta)
###generate the cellular compositions
P <- sapply(1:n, function(i){
    if(X[1,i]==0){ #if control
    rDirichlet(c(4,4, 2+X[2,i]/10))
    }else{
        rDirichlet(c(4,4, 5+X[2,i]/10))
    }
})
dim(P)#(3,180)
P[,1]

Ometh <- NULL
for(i in 1:n){
    utmp <- t(sapply(1:m, function(j){
        tmp1 <- colSums(X[ ,i] * t(beta[j, , ]))#beta(m,k,p), (2)*(2,3)
        rnorm(K,mean=mu[j, ]+tmp1,sd=0.01)#mu(2000,3)
    }))#这一部分，取定一个sample，按照paper里 o=mu+x*beta+epsilon的公式计算每一个点位的三种细胞的甲基化值.utmp(2000,3)
    dim(utmp)
    tmp2 <- colSums(P[ ,i] * t(utmp))#按P[3,180]中的构成比对每个sample取平均甲基化值
    Ometh <- cbind(Ometh, tmp2 + rnorm(m, sd = 0.01))#测量结果有误差
}
Ometh[Ometh > 1] <- 1
Ometh[Ometh < 0] <- 0
dim(Ometh)
head(Ometh)


utmp <- t(sapply(1:m, function(j){
        tmp1 <- colSums(X[ ,1] * t(beta[j, , ]))#beta(m,k,p), (2)*(2,3)
        rnorm(K,mean=mu[j, ]+tmp1,sd=0.01)#mu(2000,3)
    }))
dim(utmp)
tmp1 <- colSums(X[ ,1] * t(beta[1, , ]))
X[,1]
t(beta[1,,])
tmp1

[1] 0.2395661 0.2930486 0.6708650 0.3461865 0.3570898 0.4218902

[1] 400

[1] 400   3

0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
31.69642,42.37449,43.50825,26.22369,21.6958,29.94542,40.51416,43.03323,39.59148,48.01412,...,26.09716,36.05796,31.57542,45.86856,22.80691,21.90346,25.00292,48.06966,33.27527,40.23189


[1] 400   3   2

...

0.2177607,0.2205099,0.2227666,0.2129109,0.2074269,0.2258976,0.1974733,0.2261967,0.2075809,0.2375106,...,0.2736043,0.2472094,0.2460459,0.2522854,0.2274212,0.2605628,0.2528073,0.2541865,0.2244566,0.2659909
0.3331090,0.3282000,0.3660416,0.3554682,0.3331807,0.3089373,0.3594269,0.3435083,0.3567191,0.3589571,...,0.3806392,0.3776689,0.4952795,0.3584725,0.3120394,0.3598640,0.3532794,0.3088560,0.3252882,0.3799327
0.5358172,0.5408025,0.4915809,0.4625442,0.5330214,0.5614665,0.4721415,0.5458155,0.5492382,0.5140335,...,0.5644498,0.6012421,0.4648548,0.5921310,0.6405674,0.5805364,0.6042406,0.6341406,0.6176340,0.5662915
0.3679452,0.3558344,0.3813572,0.3726718,0.3551917,0.3613670,0.3819133,0.3588739,0.3570239,0.3797768,...,0.2633173,0.2642074,0.2545409,0.2574639,0.2445834,0.2330504,0.2383820,0.2446994,0.2449370,0.2531373
0.3380117,0.3498111,0.3426959,0.3492340,0.3504964,0.3438887,0.3198905,0.3460663,0.3422587,0.3277000,...,0.2412811,0.2618655,0.2537449,0.2743091,0.2687138,0.2817875,0.2794279,0.2523448,0.3101096,0.2440614
0.4645800,0.4592445,0.4830042,0.4949939,0.4579189,0.4676226,0.5017732,0.4781250,0.4955321,0.4901884,...,0.3712721,0.3775072,0.4094531,0.3782861,0.3783148,0.3616197,0.4051210,0.3653490,0.3772118,0.3936305


[1] 400   3

[1]  0.00000 31.69642

-0.1367069,0.08097047,0.1048167
0.0000000,0.00000000,0.0000000


[1] 0 0 0

In [347]:
HIRE<-function (Ometh, X, num_celltype, tol = 10^(-5), num_iter = 1000, 
    alpha = 0.01) 
{
    #产生随机的初始composition
    rDirichlet <- function(alpha_vec) {
        num <- length(alpha_vec)
        temp <- rgamma(num, shape = alpha_vec, rate = 1)
        return(temp/sum(temp))
    }
    #methyl matrix(2000(m),180(n))
    #哈哈哈这个学过了
    #这个函数的作用应该是估计p和u，u作为EM的初始值？
    CorDescent <- function(MethMatr, num_celltype, tol = 0.01, 
        showIter = FALSE) {
        err0 <- 0
        err1 <- 1000
        m <- nrow(MethMatr)
        n <- ncol(MethMatr)
        K <- num_celltype
        if (m < n) {
            stop("The CpgG site number must be larger than the sample number!")
        }
        P_matr_t <- vapply(seq_len(n), function(i) {
            rDirichlet(rep(2, K))#(k)
        }, FUN.VALUE = rep(-1, K))
        #P_matr_t(K,n),我猜是随机初始化,这个矩阵是要估计的composition
        while (abs(err1 - err0) >= tol) {
            err0 <- err1
            Dmat <- 2 * P_matr_t %*% t(P_matr_t)#(3,3)
            Amat <- cbind(diag(rep(1, K)), diag(rep(-1, K)))#(3,6)
            bvec <- c(rep(0, K), rep(-1, K))#(1,6)
            U_matr_t <- t(vapply(seq_len(m), function(j) {
                dvec <- 2 * P_matr_t %*% as.numeric(MethMatr[j, 
                  ])#column(3),每个元素对应估计的composition导致的该种细胞的总methylation值(对每一个点位估计)
                solu <- solve.QP(Dmat, dvec, Amat, bvec)
                solu$solution
            }, FUN.VALUE = rep(-1, K)))
            ###  U(2000,3), solu(3),这里的限制条件是solution的每个值大于等于0小于等于1，
            ###  求解的是min b*P_matr*t(P_matr)*t(b)-2*t(dev)*b,这里的每一个解b都对应一个site的三种细胞的不知道是啥(我猜是composition)
            ###猜测U估计的是每个点位三种细胞类型的methyl值
            Dmat <- 2 * t(U_matr_t) %*% U_matr_t#(3,3)
            Amat <- cbind(matrix(1, K, K), diag(rep(1, K)))
            bvec <- c(rep(1, K), rep(0, K))
            P_matr_t <- vapply(seq_len(n), function(i) {
                dvec <- 2 * t(U_matr_t) %*% as.numeric(MethMatr[, 
                  i])#column(3),每个type对应估计的composition导致的该种细胞的总methylation值(对每一个细胞估计)
                solu <- solve.QP(Dmat, dvec, Amat, bvec, meq = K)
                solu$solution
            }, FUN.VALUE = rep(-1, K))
            ### 这里的条件是x+y+z>=1,x,y,z>=0，解(3,180)对应
            
            
            
            err1 <- sum((MethMatr - U_matr_t %*% P_matr_t)^2)
            if (showIter == TRUE) {
                message("  ", err1, "\n")
            }
        }
        return(list(U = U_matr_t, P = P_matr_t))
    }
    
    
    Initialize <- function(Ometh, num_celltype) {
        K <- num_celltype
        sdrow <- apply(Ometh, 1, sd)#对Ometh(2000,180)每一行求sd，每一行对应一个site
        ind <- order(sdrow, decreasing = TRUE)#排个序
        m <- nrow(Ometh)
        if (m <= 1000) {
            num_cpg_for_init <- m
        }
        else {
            num_cpg_for_init <- max(3 * ncol(Ometh), floor(m/10))#floor盲猜取整，限制m的值，目的不明(应该是方便计算，因为后面只用到了p的估计，p的估计维度不受m的影响)
            if (num_cpg_for_init > m) {
                num_cpg_for_init <- m
            }
        }
        Ometh_part <- Ometh[ind[seq_len(num_cpg_for_init)], ]#取前多少个。。目的不明
        result <- CorDescent(Ometh_part, num_celltype = K, tol = 0.1, 
            showIter = FALSE)
        P_initial <- result$P
        mu_initial <- vapply(seq_len(m), function(j) {
            if (K > 2) {
                fit <- lm(Ometh[j, ] ~ as.matrix(t(P_initial[-1, 
                  ])))#as.,atrix作为(2,180)矩阵返回,n加t之后是(180,2)
            }
            ##P[-1,]意思是舍掉第一行，结果为(2,180)，(为啥舍掉一行也没搞明白)
            else {
                fit <- lm(Ometh[j, ] ~ as.numeric(P_initial[-1, 
                  ]))#as.numeric作为向量返回
            }
            tmp <- as.numeric(summary(fit)$coeff[, 1])##coeff的具体信息打在下面了
            tmp[-1] <- tmp[1] + tmp[-1]#[-1]是不带截距，+[1]是补正截距，目的未知
            tmp#这个tmp估计的应该是mu_k(每个cell type的baseline)之类的值，但是上面的算法还没看懂
        }, FUN.VALUE = rep(-1, K))
        ##噫，我看懂了，输出的mu(mu1,mu2,mu3)是对每种type mu的估计，别问为什么
        
        return(list(P_initial, mu_initial))#mu(3,2000)
    }
   
    #我要开始大胆的尝试了
    #我觉得我应该先写一个R，没问题的话再换C++
    #好那么我要开始了
    ##初始版本
    my_EM=function(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter){
        error0=0
        error1=1000
        n=ncol(Ometh)
        m=nrow(Ometh)
        K=nrow(P_t)
        q=nrow(X)#phenotype
        #t表示循环子
        #我们规定每次更新不保留前值
        #Ometh (m,n), X (q,n), P_t (K,n), mu_t (m,K), beta_t (m,K,q), sigma1,sigma2 scalar
        #sigma_ijt (n,K,K)-(i,...) 本来应该是 (n,m K,K), 但我发现share variance的情况好像与j无关
        #算了我还是把j加上吧。。要不不好写 sigma_ijt (n,m,K,K)-(i,j,...)
        #mu_ijt (n,m,K) -(i,j,...)  column
        sigma_ijt=array(rep(0,n*m*K*K),dim=c(n,m,K,K))
        mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))
        
        for(t in 1:num_iter){
            
            for(i in 1:n){
                sum_pit_sq=sum(P_t[,i]*P_t[,i])
                for(j in 1:m){
                    #首先要算sigma_ij^t和 mu_ij^t
                    sigma_ijt[i, j, , ]=diag(sigma1**2, K, K)-(sigma1**4)*(P_t[, i]%*%t(P_t[, i])/((sigma2**2)+sum_pit_sq*sigma1**2))
                    mu_ijt[i, j, ]=t((Ometh[j,i]*t(P_t[,i])/(sigma2**2)+t(mu_t[j,]+beta_t[j,,]%*%X[,i])/sigma1**2)%*%sigma_ijt[i, j, , ])    
                }
                j=1
            }#upto here we have calculated all mu_ijt and sigma_ijt for this iteration
            #update mu_t[j, ] its really slow...
            i=1
            for(j_mu in 1:m){
                sum_respect_to_n=colSums(mu_ijt[, j_mu, ])
                numerator_mu=sum_respect_to_n-rowSums(beta_t[j_mu, , ]%*%X)
                mu_t[j_mu, ]=numerator_mu/n
            }
            j_mu=1
            #update beta..., 对j&l循环
            for(j_beta in 1:m){
                for(l_beta in 1:q){
                    y_jl=vapply(seq_len(n), function(i_beta){
                        mu_ijt[i_beta, j_beta, ]-mu_t[j_beta, ]-beta_t[j_beta, , ]%*%X[, i_beta]+beta_t[j_beta, ,l_beta]*X[l_beta,i_beta]
                    }, FUN.VALUE=rep(0,K))
                    numerator_beta=y_jl%*%(X[l_beta, ])
                    beta_t[j_beta, ,l_beta]=numerator_beta/(sum(X[l_beta, ]*X[l_beta, ]))
                }
                l_beta=1
            }
            j_beta=1
            #update p这个先等等。。
            #update sigma1
            numerator_sigma1=0
            for(i_sigma1 in 1:n){
                B_i=vapply(seq_len(m), function(j_sigma1){
                    mu_ijt[i_sigma1, j_sigma1, ]-mu_t[j_sigma1, ]-beta_t[j_sigma1, , ]%*%X[,i_sigma1]
                }, FUN.VALUE=rep(0,K))
                numerator_sigma1=numerator_sigma1+sum(diag(t(B_i)%*%B_i))+m*sum(diag(sigma_ijt[i_sigma1, 1, , ]))
            }
            i_sigma1=1
            sigma1_square=numerator_sigma1/(m*n*K)
            sigma1=sqrt(sigma1_square)
            #update sigma2
            temp=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, 1, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
            #还要一个对i循环
            for(i_sigma2_1 in 1:n){
                temp=temp+(1/(m*n))*sum(vapply(seq_len(m), function(j){(Ometh[j, i_sigma2_1]-t(mu_ijt[i_sigma2_1, j, ])%*%P_t[, i_sigma2_1])**2}, FUN.VALUE=rep(1,1)))
            }
            i_sigma2_1=1
            sigma2=sqrt(temp)
            #update P_t
            Amat=cbind(matrix(1, K, K), diag(rep(1, K)))
            bvec=c(rep(1, K), rep(0, K))
            P_t=vapply(seq_len(n), function(i){
                Dmat=2*(m*sigma_ijt[i, 1, , ]+t(mu_ijt[i, , ])%*%mu_ijt[i, , ])
                dvec=2*t(colSums(Ometh[ , i]*mu_ijt[i, , ]))
                solu=solve.QP(Dmat, dvec, Amat, bvec, meq = K)
                return(solu$solution)
            }, FUN.VALUE=rep(1, K))
            
            
        }
        ret_list=list("P_t"=P_t, "mu_t"=mu_t, "beta_t"=beta_t, "sigma1"=sigma1, "sigma2"=sigma2)
        return(ret_list)
    }
    #优化一下计算时间
    my_EM_1=function(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter){
        error0=0
        error1=1000
        n=ncol(Ometh)
        m=nrow(Ometh)
        K=nrow(P_t)
        q=nrow(X)#phenotype
        #t表示循环子
        #我们规定每次更新不保留前值
        #Ometh (m,n), X (q,n), P_t (K,n), mu_t (m,K), beta_t (m,K,q), sigma1,sigma2 scalar
        #sigma_ijt (n,K,K)-(i,...) 本来应该是 (n,m K,K), 但我发现share variance的情况好像与j无关
        
        #mu_ijt (n,m,K) -(i,j,...)  column
        sigma_ijt=array(rep(0,n*K*K),dim=c(n,K,K))
        mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))
        
        for(t in 1:num_iter){
            
            sum_pit_sq=vapply(seq_len(n), function(i){sum(P_t[, i]*P_t[, i])}, rep(1,1))
            for(i in 1:n){sigma_ijt[i, , ]=diag(sigma1**2, K, K)-(sigma1**4)*(P_t[, i]%*%t(P_t[, i])/((sigma2**2)+sum_pit_sq[i]*sigma1**2))}
            i=1
            for(i in 1:n){
                mu_ijt[i, , ]=vapply(seq_len(m), function(j){
                    t((Ometh[j,i]*t(P_t[,i])/(sigma2**2)+t(mu_t[j,]+beta_t[j,,]%*%X[,i])/sigma1**2)%*%sigma_ijt[i, , ]) 
                }, FUN.VALUE=rep(1, K))
            }
            i=1
            
            
            #update mu_t[j, ] 
            mu_t=t(vapply(seq_len(m), function(j){
               (colSums(mu_ijt[, j, ])-rowSums(beta_t[j, , ]%*%X))/n
            }, FUN.VALUE=rep(1,K)))
            
            #update beta..., 对j&l循环
            for(j_beta in 1:m){
                for(l_beta in 1:q){
                    y_jl=vapply(seq_len(n), function(i_beta){
                        mu_ijt[i_beta, j_beta, ]-mu_t[j_beta, ]-beta_t[j_beta, , ]%*%X[, i_beta]+beta_t[j_beta, ,l_beta]*X[l_beta,i_beta]
                    }, FUN.VALUE=rep(0,K))
                    numerator_beta=y_jl%*%(X[l_beta, ])
                    beta_t[j_beta, ,l_beta]=numerator_beta/(sum(X[l_beta, ]*X[l_beta, ]))
                }
                l_beta=1
            }
            j_beta=1
            #update p这个先等等。。
            #update sigma1
            numerator_sigma1=0
            for(i_sigma1 in 1:n){
                B_i=vapply(seq_len(m), function(j_sigma1){
                    mu_ijt[i_sigma1, j_sigma1, ]-mu_t[j_sigma1, ]-beta_t[j_sigma1, , ]%*%X[,i_sigma1]
                }, FUN.VALUE=rep(0,K))
                numerator_sigma1=numerator_sigma1+sum(diag(t(B_i)%*%B_i))+m*sum(diag(sigma_ijt[i_sigma1, , ]))
            }
            i_sigma1=1
            sigma1_square=numerator_sigma1/(m*n*K)
            sigma1=sqrt(sigma1_square)
            #update sigma2
            temp=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
            #还要一个对i循环
            for(i_sigma2_1 in 1:n){
                temp=temp+(1/(m*n))*sum(vapply(seq_len(m), function(j){(Ometh[j, i_sigma2_1]-t(mu_ijt[i_sigma2_1, j, ])%*%P_t[, i_sigma2_1])**2}, FUN.VALUE=rep(1,1)))
            }
            i_sigma2_1=1
            sigma2=sqrt(temp)
            #update P_t
            Amat=cbind(matrix(1, K, K), diag(rep(1, K)))
            bvec=c(rep(1, K), rep(0, K))
            P_t=vapply(seq_len(n), function(i){
                Dmat=2*(m*sigma_ijt[i, , ]+t(mu_ijt[i, , ])%*%mu_ijt[i, , ])
                dvec=2*t(colSums(Ometh[ , i]*mu_ijt[i, , ]))
                solu=solve.QP(Dmat, dvec, Amat, bvec, meq = K)
                return(solu$solution)
            }, FUN.VALUE=rep(1, K))
            
            
        }
        ret_list=list("P_t"=P_t, "mu_t"=mu_t, "beta_t"=beta_t, "sigma1"=sigma1, "sigma2"=sigma2)
        return(ret_list)
        
    }
    
    
    
    m <- nrow(Ometh)
    n <- ncol(Ometh)
    p <- nrow(X)#X phenotype,(2,180)
    K <- num_celltype
    P_t <- matrix(-1, K, n)
    mu_t <- matrix(-1, m, K)
    beta_t <- array(-1, dim = c(m, K, p))
    sig_sqTiss_t <- matrix(-1, m, K)
    sig_sqErr_t <- rep(-1, m)
    init <- Initialize(Ometh, K)
    P_t <- init[[1]]
    mu_t <- t(init[[2]])
    message("  Initialization Done.\n")
    message("  Implementing EM algorithm... \n")
    ret_list <- my_EM(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter)
    message("  Done! \n")
    message("  Calculating p-values...\n")
    tmp <- NULL
    for (ell in seq_len(p)) {
        tmp <- cbind(tmp, X[ell, ] * t(ret_list$P_t))#注意不是矩阵乘法
    }#(180,6)不知道具体是啥。。。每种细胞对表现型的贡献。。？
    x_matr <- cbind(tmp, t(ret_list$P_t)[, seq(2, K)])#(180,8)
    x_matr <- as.matrix(x_matr)
    pvalues <- t(vapply(seq_len(m), function(j) {
        y_vec <- Ometh[j, ]
        fit <- lm(y_vec ~ x_matr)
        summary(fit)$coef[seq(2, (1 + p * K)), 4]#第四列是p，第一行是截距，后六行对应斜率，大概吧
    }, FUN.VALUE = rep(-1, p * K)))#p(2000,6)判断的是每种cell type在每个点位受不受表现型影响
    message("  Done!\n")
    ret_list[[7]] <- pvalues
    names(ret_list)[7] <- "pvalues"
    names(ret_list)[6] <- "pBIC"
    d <- (K - 1) * n + m * (1 + 2 * K + p * K)
    d0 <- sum(ret_list$pvalues > alpha/(p * m * K))
    ret_list[[6]] <- ret_list[[6]] - log(n) * d + log(n) * (d - 
        d0)#这个BIC是啥
    return(ret_list)
}

In [348]:
n=90
m=400
K=3
q=2
sigma_ijt=array(rep(0,n*m*K*K),dim=c(n,m,K,K))
mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))


P_t <- P
mu_t <- mu+rnorm(1,0,0.05)
beta_t <- beta
sigma1=0.01
sigma2=0.01

num_iter=1

In [349]:
head(mu-mu_t)


methy1,methy2,methy3
-0.02246924,-0.02246924,-0.02246924
-0.02246924,-0.02246924,-0.02246924
-0.02246924,-0.02246924,-0.02246924
-0.02246924,-0.02246924,-0.02246924
-0.02246924,-0.02246924,-0.02246924
-0.02246924,-0.02246924,-0.02246924


In [270]:
head(mu_t-mu)
sigma1
sigma2
P_t-P
#mu_t1=mu_t

methy1,methy2,methy3
-0.02510116,-0.02637581,-0.02116242
-0.02477803,-0.02580951,-0.02060070
-0.02542195,-0.02637091,-0.02145322
-0.02532268,-0.02629294,-0.02144880
-0.02482325,-0.02623105,-0.02110520
-0.02449087,-0.02574684,-0.02048388


[1] 0.01047597

[1] 0.02553332

-0.004034622,1.127624e-04,-0.003715118,0.005097269,0.003240324,-0.0064255576,0.0014158188,0.0002426675,-0.0006909772,-0.005747239,...,-0.014319191,-0.002803223,-0.0032428696,-0.004044024,-0.0048466631,-0.009918763,-0.003341843,0.01063223,0.0003224988,-0.002703926
0.009486194,5.118316e-05,0.007381314,-0.011363560,-0.007231084,0.0068457369,-0.0005772185,0.0003041528,0.0021822946,0.008533722,...,0.010221453,0.005778896,0.0028550228,0.007237288,-0.0004533229,0.004457690,-0.001717879,-0.00495322,-0.0015317861,0.006317731
-0.005451572,-1.639455e-04,-0.003666196,0.006266291,0.003990760,-0.0004201793,-0.0008386003,-0.0005468203,-0.0014913174,-0.002786483,...,0.004097738,-0.002975674,0.0003878467,-0.003193264,0.0052999860,0.005461073,0.005059722,-0.00567901,0.0012092873,-0.003613804


In [318]:
system.time(my_EM(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter))
system.time(my_EM_1(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter))
ret1=my_EM(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter)
ret2=my_EM_1(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter)

   user  system elapsed 
   1.75    0.00    1.75 

   user  system elapsed 
   1.39    0.04    1.36 

In [319]:
sum(ret1$mu_t-ret2$mu_t)


[1] -3.191891e-16

0.3294782,0.3980104,0.3368452,0.36145042,0.3928957,0.4034509,0.2639779,0.4097859,0.3597392,0.2195224,...,0.1087809,0.1684273,0.13740282,0.2545325,0.2656785,0.2211227,0.2506511,0.1489554,0.3946224,0.1848559
0.3334558,0.2883336,0.1965358,0.07016961,0.2760739,0.3449841,0.1920797,0.2644601,0.3114712,0.3750009,...,0.3621480,0.3389119,0.06691263,0.2397395,0.3706887,0.3045381,0.2841538,0.5087312,0.1956696,0.3313702
0.3370660,0.3136560,0.4666190,0.56837998,0.3310304,0.2515650,0.5439424,0.3257540,0.3287896,0.4054767,...,0.5290711,0.4926608,0.79568455,0.5057280,0.3636328,0.4743392,0.4651952,0.3423134,0.4097080,0.4837739


In [353]:
sigma_ijt=array(rep(0,n*K*K),dim=c(n,K,K))
mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))

P_t <- P
mu_t <- mu
beta_t <- beta
sigma1=0.01
sigma2=0.01

temp=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
            #还要一个对i循环
            for(i_sigma2_1 in 1:n){
                temp=temp+(1/(m*n))*sum(vapply(seq_len(m), function(j){(Ometh[j, i_sigma2_1]-t(mu_ijt[i_sigma2_1, j, ])%*%P_t[, i_sigma2_1])**2}, FUN.VALUE=rep(1,1)))
            }
            i_sigma2_1=1
            sigma2=sqrt(temp)
temp1=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
temp2=1/(m*n)*sum(vapply(seq_len(n), function(i){
    sum(vapply(seq_len(m), function(j){(Ometh[j, i]-t(mu_ijt[i, j, ])%*%P_t[, i])**2}, FUN.VALUE=rep(1,1)))
}, FUN.VALUE=rep(1,1)))
sigma22=sqrt(temp1+temp2)

In [354]:

sigma22
sigma2


[1] 0.3473103

[1] 0.3473103

In [317]:
 my_EM=function(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter){
        error0=0
        error1=1000
        n=ncol(Ometh)
        m=nrow(Ometh)
        K=nrow(P_t)
        q=nrow(X)#phenotype
        #t表示循环子
        #我们规定每次更新不保留前值
        #Ometh (m,n), X (q,n), P_t (K,n), mu_t (m,K), beta_t (m,K,q), sigma1,sigma2 scalar
        #sigma_ijt (n,K,K)-(i,...) 本来应该是 (n,m K,K), 但我发现share variance的情况好像与j无关
        #算了我还是把j加上吧。。要不不好写 sigma_ijt (n,m,K,K)-(i,j,...)
        #mu_ijt (n,m,K) -(i,j,...)  column
        sigma_ijt=array(rep(0,n*m*K*K),dim=c(n,m,K,K))
        mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))
        
        for(t in 1:num_iter){
            
            for(i in 1:n){
                sum_pit_sq=sum(P_t[,i]*P_t[,i])
                for(j in 1:m){
                    #首先要算sigma_ij^t和 mu_ij^t
                    sigma_ijt[i, j, , ]=diag(sigma1**2, K, K)-(sigma1**4)*(P_t[, i]%*%t(P_t[, i])/((sigma2**2)+sum_pit_sq*sigma1**2))
                    mu_ijt[i, j, ]=t((Ometh[j,i]*t(P_t[,i])/(sigma2**2)+t(mu_t[j,]+beta_t[j,,]%*%X[,i])/sigma1**2)%*%sigma_ijt[i, j, , ])    
                }
                j=1
            }#upto here we have calculated all mu_ijt and sigma_ijt for this iteration
            #update mu_t[j, ] its really slow...
            i=1
            for(j_mu in 1:m){
                sum_respect_to_n=colSums(mu_ijt[, j_mu, ])
                numerator_mu=sum_respect_to_n-rowSums(beta_t[j_mu, , ]%*%X)
                mu_t[j_mu, ]=numerator_mu/n
            }
            j_mu=1
            #update beta..., 对j&l循环
            for(j_beta in 1:m){
                for(l_beta in 1:q){
                    y_jl=vapply(seq_len(n), function(i_beta){
                        mu_ijt[i_beta, j_beta, ]-mu_t[j_beta, ]-beta_t[j_beta, , ]%*%X[, i_beta]+beta_t[j_beta, ,l_beta]*X[l_beta,i_beta]
                    }, FUN.VALUE=rep(0,K))
                    numerator_beta=y_jl%*%(X[l_beta, ])
                    beta_t[j_beta, ,l_beta]=numerator_beta/(sum(X[l_beta, ]*X[l_beta, ]))
                }
                l_beta=1
            }
            j_beta=1
            #update p这个先等等。。
            #update sigma1
            numerator_sigma1=0
            for(i_sigma1 in 1:n){
                B_i=vapply(seq_len(m), function(j_sigma1){
                    mu_ijt[i_sigma1, j_sigma1, ]-mu_t[j_sigma1, ]-beta_t[j_sigma1, , ]%*%X[,i_sigma1]
                }, FUN.VALUE=rep(0,K))
                numerator_sigma1=numerator_sigma1+sum(diag(t(B_i)%*%B_i))+m*sum(diag(sigma_ijt[i_sigma1, 1, , ]))
            }
            i_sigma1=1
            sigma1_square=numerator_sigma1/(m*n*K)
            sigma1=sqrt(sigma1_square)
            #update sigma2
            temp=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, 1, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
            #还要一个对i循环
            for(i_sigma2_1 in 1:n){
                temp=temp+(1/(m*n))*sum(vapply(seq_len(m), function(j){(Ometh[j, i_sigma2_1]-t(mu_ijt[i_sigma2_1, j, ])%*%P_t[, i_sigma2_1])**2}, FUN.VALUE=rep(1,1)))
            }
            i_sigma2_1=1
            sigma2=sqrt(temp)
            #update P_t
            Amat=cbind(matrix(1, K, K), diag(rep(1, K)))
            bvec=c(rep(1, K), rep(0, K))
            P_t=vapply(seq_len(n), function(i){
                Dmat=2*(m*sigma_ijt[i, 1, , ]+t(mu_ijt[i, , ])%*%mu_ijt[i, , ])
                dvec=2*t(colSums(Ometh[ , i]*mu_ijt[i, , ]))
                solu=solve.QP(Dmat, dvec, Amat, bvec, meq = K)
                return(solu$solution)
            }, FUN.VALUE=rep(1, K))
            
            
        }
     
     ret_list=list("P_t"=P_t, "mu_t"=mu_t, "beta_t"=beta_t, "sigma1"=sigma1, "sigma2"=sigma2)
        return(ret_list)
    }
    #优化一下计算时间
    my_EM_1=function(Ometh, X, P_t, mu_t, beta_t, sigma1, sigma2, tol, num_iter){
        error0=0
        error1=1000
        n=ncol(Ometh)
        m=nrow(Ometh)
        K=nrow(P_t)
        q=nrow(X)#phenotype
        #t表示循环子
        #我们规定每次更新不保留前值
        #Ometh (m,n), X (q,n), P_t (K,n), mu_t (m,K), beta_t (m,K,q), sigma1,sigma2 scalar
        #sigma_ijt (n,K,K)-(i,...) 本来应该是 (n,m K,K), 但我发现share variance的情况好像与j无关
        
        #mu_ijt (n,m,K) -(i,j,...)  column
        sigma_ijt=array(rep(0,n*K*K),dim=c(n,K,K))
        mu_ijt=array(rep(0,n*m*K),dim=c(n,m,K))
        
        for(t in 1:num_iter){
            
            sum_pit_sq=vapply(seq_len(n), function(i){sum(P_t[, i]*P_t[, i])}, rep(1,1))
            for(i in 1:n){sigma_ijt[i, , ]=diag(sigma1**2, K, K)-(sigma1**4)*(P_t[, i]%*%t(P_t[, i])/((sigma2**2)+sum_pit_sq[i]*sigma1**2))}
            i=1
            for(i in 1:n){
                mu_ijt[i, , ]=vapply(seq_len(m), function(j){
                    t((Ometh[j,i]*t(P_t[,i])/(sigma2**2)+t(mu_t[j,]+beta_t[j,,]%*%X[,i])/sigma1**2)%*%sigma_ijt[i, , ]) 
                }, FUN.VALUE=rep(1, K))
            }
            i=1
            
            
            #update mu_t[j, ] 
            mu_t=t(vapply(seq_len(m), function(j){
               (colSums(mu_ijt[, j, ])-rowSums(beta_t[j, , ]%*%X))/n
            }, FUN.VALUE=rep(1,K)))
            
            #update beta..., 对j&l循环
            for(j_beta in 1:m){
                for(l_beta in 1:q){
                    y_jl=vapply(seq_len(n), function(i_beta){
                        mu_ijt[i_beta, j_beta, ]-mu_t[j_beta, ]-beta_t[j_beta, , ]%*%X[, i_beta]+beta_t[j_beta, ,l_beta]*X[l_beta,i_beta]
                    }, FUN.VALUE=rep(0,K))
                    numerator_beta=y_jl%*%(X[l_beta, ])
                    beta_t[j_beta, ,l_beta]=numerator_beta/(sum(X[l_beta, ]*X[l_beta, ]))
                }
                l_beta=1
            }
            j_beta=1
            #update p这个先等等。。
            #update sigma1
            numerator_sigma1=0
            for(i_sigma1 in 1:n){
                B_i=vapply(seq_len(m), function(j_sigma1){
                    mu_ijt[i_sigma1, j_sigma1, ]-mu_t[j_sigma1, ]-beta_t[j_sigma1, , ]%*%X[,i_sigma1]
                }, FUN.VALUE=rep(0,K))
                numerator_sigma1=numerator_sigma1+sum(diag(t(B_i)%*%B_i))+m*sum(diag(sigma_ijt[i_sigma1, , ]))
            }
            i_sigma1=1
            sigma1_square=numerator_sigma1/(m*n*K)
            sigma1=sqrt(sigma1_square)
            #update sigma2
            temp=(1/n)*sum(vapply(seq_len(n), function(i_sigma2){t(P_t[, i_sigma2])%*%sigma_ijt[i_sigma2, , ]%*%P_t[, i_sigma2]}, FUN.VALUE=rep(1,1)))#sigma_ijt j取1即可
            #还要一个对i循环
            for(i_sigma2_1 in 1:n){
                temp=temp+(1/(m*n))*sum(vapply(seq_len(m), function(j){(Ometh[j, i_sigma2_1]-t(mu_ijt[i_sigma2_1, j, ])%*%P_t[, i_sigma2_1])**2}, FUN.VALUE=rep(1,1)))
            }
            i_sigma2_1=1
            sigma2=sqrt(temp)
            #update P_t
            Amat=cbind(matrix(1, K, K), diag(rep(1, K)))
            bvec=c(rep(1, K), rep(0, K))
            P_t=vapply(seq_len(n), function(i){
                Dmat=2*(m*sigma_ijt[i, , ]+t(mu_ijt[i, , ])%*%mu_ijt[i, , ])
                dvec=2*t(colSums(Ometh[ , i]*mu_ijt[i, , ]))
                solu=solve.QP(Dmat, dvec, Amat, bvec, meq = K)
                return(solu$solution)
            }, FUN.VALUE=rep(1, K))
            
            
        }
       ret_list=list("P_t"=P_t, "mu_t"=mu_t, "beta_t"=beta_t, "sigma1"=sigma1, "sigma2"=sigma2)
        return(ret_list)
    }